In [1]:
import pandas as pd
import numpy as np
import scipy

prec = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/prec_1991_2015.xls')
temp = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/temp_1991_2015.xls')
occur = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/aegypti_albopictus.xlsx')

In [2]:
# Table of occurrences
occur.head()

,VECTOR,OCCURRENCE_ID,SOURCE_TYPE,LOCATION_TYPE,POLYGON_ADMIN,Y,X,YEAR,COUNTRY,COUNTRY_ID,GAUL_AD0,STATUS
0,Aedes aegypti,1,published,point,-999,-3.22,40.07,1958,Kenya,KEN,133,NaN
1,Aedes aegypti,2,published,point,-999,-4.27,15.30,1960,Congo,COG,59,NaN
2,Aedes aegypti,3,published,point,-999,-4.27,15.30,1960,Congo,COG,59,NaN
3,Aedes aegypti,4,published,point,-999,-3.22,40.07,1960,Kenya,KEN,133,NaN
4,Aedes aegypti,5,published,point,-999,-3.04,40.14,1960,Kenya,KEN,133,NaN


In [3]:
# Table of temperature
temp.head()

,tas,Year,Month,Country,ISO3,ISO2
0,25.3234,1991,1,MDG,NaN,NaN
1,24.8328,1991,2,MDG,NaN,NaN
2,24.6455,1991,3,MDG,NaN,NaN
3,23.4024,1991,4,MDG,NaN,NaN
4,21.7749,1991,5,MDG,NaN,NaN


In [4]:
# Table of precipitation
prec.head()

,pr,Year,Month,Country,ISO3,ISO2
0,127.9060,1991,1,LBN,NaN,NaN
1,67.7115,1991,2,LBN,NaN,NaN
2,140.4620,1991,3,LBN,NaN,NaN
3,47.5851,1991,4,LBN,NaN,NaN
4,15.3227,1991,5,LBN,NaN,NaN


In [5]:
# Rename the columns to match among themselves
temp.rename(columns={'\tYear':'Year', ' Month':'Month', ' Country':'Country'}, inplace=True)
prec.rename(columns={'\tYear':'Year', ' Month':'Month', ' Country':'Country'}, inplace=True)
occur.rename(columns={'YEAR':'Year', 'COUNTRY_ID':'Country'}, inplace=True)

In [6]:
# Add Taywan to dataset because the original source of temperatures and rainfall 
# doesn't contain this country. Taywan has several occurrences, so it was crutial
# to look for its data. 
taywan_temp = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/TWN_data.xlsx', sheet_name='Temp')
taywan_rain = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/TWN_data.xlsx', sheet_name='Rainfall')
taywan= pd.merge(taywan_temp, taywan_rain, how='left', left_on=['Country','Year'], right_on = ['Country','Year'])
taywan.rename(columns={'temp':'tas'}, inplace=True)
taywan = taywan.loc[(taywan['Year']>=1991) & (taywan['Year']<=2015)].reset_index(drop=True)
taywan

,tas,Year,Country,pr
0,22.1,1991,TWN,2032.2
1,22.0,1992,TWN,2199.9
2,21.7,1993,TWN,1671.9
3,22.0,1994,TWN,2432.7
4,21.7,1995,TWN,1639.1
5,22.1,1996,TWN,2608.1
6,21.6,1997,TWN,2117.9
7,22.2,1998,TWN,2103.1
8,22.2,1999,TWN,2658.3
9,21.5,2000,TWN,2417.4


In [7]:
# Create a file with the merged data of Taywan
taywan.to_csv('taywan.csv')

In [8]:
# Read the Taywan table
taywan = pd.read_csv('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/taywan.csv')
taywan.drop(columns=['Unnamed: 0'],inplace=True)
taywan.shape

(25, 4)

## Preprocessing of Occurrences

In [9]:
# Drop unnecessary columns for the analyzes
occur.drop(columns=['SOURCE_TYPE','LOCATION_TYPE','POLYGON_ADMIN','STATUS','GAUL_AD0','X','Y'],inplace=True)
occur.head()

,VECTOR,OCCURRENCE_ID,Year,COUNTRY,Country
0,Aedes aegypti,1,1958,Kenya,KEN
1,Aedes aegypti,2,1960,Congo,COG
2,Aedes aegypti,3,1960,Congo,COG
3,Aedes aegypti,4,1960,Kenya,KEN
4,Aedes aegypti,5,1960,Kenya,KEN


In [10]:
# Change the year type from string to int
occur['Year'] = pd.to_numeric(occur['Year'], downcast='integer',errors='coerce')
occur.describe()

,OCCURRENCE_ID,Year
count,42066.000000,40804.000000
mean,21033.500000,2006.723630
std,12143.552549,7.616601
min,1.000000,1958.000000
25%,10517.250000,2004.000000
50%,21033.500000,2008.000000
75%,31549.750000,2013.000000
max,42066.000000,2014.000000


In [11]:
# Select the timeframe of temperature and rainfall we colected 
occur = occur.loc[(occur['Year']>=1991) & (occur['Year']<=2015)]

# Set as NaN the countries we didn't find the temperature nor the rainfall
occur['Country'] = occur['Country'].replace('XXX', np.nan)
occur['Country'] = occur['Country'].replace('HKG', np.nan)

occur['Year'] = occur['Year'].astype(int)
occur.describe()

,OCCURRENCE_ID,Year
count,39172.000000,39172.000000
mean,21502.137113,2007.862734
std,11715.914535,4.979114
min,1069.000000,1991.000000
25%,11778.750000,2005.000000
50%,21571.500000,2008.000000
75%,31608.250000,2013.000000
max,42042.000000,2014.000000


In [12]:
# Aggregate the occurrences by country in an annual basis
annualOccur = pd.DataFrame(occur.groupby(['Year','Country'])['OCCURRENCE_ID'].count()).reset_index()
annualOccur.shape

(632, 3)

## Preprocessing of Temperature and Rainfall

In [13]:
# Aggregate the occurrences by country in an annual basis without Taywan
annualTemp = pd.DataFrame(temp.groupby(['Year','Country'])['tas'].sum()/12).reset_index()
annualPrec = pd.DataFrame(prec.groupby(['Year','Country'])['pr'].sum()/12).reset_index()
temp_rain=pd.merge(annualTemp,annualPrec,
                   how='inner',
                   left_on = ['Year','Country'],
                   right_on = ['Year','Country'])
temp_rain.describe()

,Year,tas,pr
count,3650.000000,3650.000000,3650.000000
mean,2003.000000,23.490532,137.480884
std,7.212091,14.660920,89.987102
min,1991.000000,-7.018926,4.440862
25%,1997.000000,19.656523,75.091654
50%,2003.000000,24.988167,124.989354
75%,2009.000000,26.483633,185.351331
max,2015.000000,187.946183,801.706467


## Merge of Temperature and Rainfall (Main Table)

In [14]:
# Add Taywan to the dataframe
temp_rain = pd.concat([temp_rain, taywan])

# Different abbreviation of the same country in different datasets
temp_rain['Country'] = temp_rain['Country'].replace('ROM', 'ROU')
temp_rain

/Users/RodolfoSaldanha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


,Country,Year,pr,tas
0,ABW,1991,38.882968,28.078008
1,AFG,1991,36.931453,13.118033
2,AGO,1991,83.844948,21.652200
3,AIA,1991,113.631158,26.108292
4,ALB,1991,77.587033,11.038935
5,ARG,1991,52.792958,14.416431
6,ASM,1991,263.307917,26.879967
7,ATG,1991,183.339717,26.413642
8,AUS,1991,37.912953,22.172075
9,AUT,1991,94.343317,6.151395


## Merge Occurrences to the Main Table

In [15]:
temp_rain_occur=pd.merge(temp_rain,annualOccur,
                   how='left',
                   left_on = ['Year','Country'],
                   right_on = ['Year','Country'])
temp_rain_occur.rename(columns={'OCCURRENCE_ID':'Ocurrences', 'tas':'Temperature', 'pr':'Precipitation'}, inplace=True)
temp_rain_occur.shape

(3675, 5)

In [16]:
temp_rain_occur.describe()

,Year,Precipitation,Temperature,Ocurrences
count,3675.000000,3675.000000,3675.000000,632.000000
mean,2003.000000,150.946483,23.479929,61.971519
std,7.212084,187.992485,14.611533,430.992062
min,1991.000000,4.440862,-7.018926,1.000000
25%,1997.000000,75.340175,19.721479,1.000000
50%,2003.000000,125.000000,24.962717,4.000000
75%,2009.000000,186.939941,26.450129,12.000000
max,2015.000000,2744.000000,187.946183,5527.000000


In [17]:
# Fill the the year in which no occurrences happened in the countries
temp_rain_occur.fillna(0,inplace=True)
temp_rain_occur.head()

,Country,Year,Precipitation,Temperature,Ocurrences
0,ABW,1991,38.882968,28.078008,0.0
1,AFG,1991,36.931453,13.118033,0.0
2,AGO,1991,83.844948,21.652200,0.0
3,AIA,1991,113.631158,26.108292,0.0
4,ALB,1991,77.587033,11.038935,0.0


In [18]:
# File containing temperature, rainfall and occurrences
temp_rain_occur.to_csv('occurrences.csv', encoding='utf-8', index=False)

## Preprocessing of Urbanization

In [20]:
# Dataset about urban and rural population percentage
urb = pd.read_csv('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/urban.csv')
urb.rename(columns={'Code':'Country'}, inplace=True)
urb.describe()

,Year,Urban (%),Rural (%)
count,27977.000000,27977.000000,27977.000000
mean,1996.039139,53.575697,46.424303
std,46.798435,26.745724,26.745724
min,1500.000000,0.000000,0.000000
25%,1974.000000,31.712000,24.386000
50%,1999.000000,53.767149,46.232853
75%,2025.000000,75.614000,68.288002
max,2050.000000,100.000000,100.000000


In [21]:
# Check for countries of the main Dataframe that aren't in the urbanization dataset
countryList = urb['Country'].unique()
temp_rain_occur[~temp_rain_occur['Country'].isin(countryList)]

,Country,Year,Precipitation,Temperature,Ocurrences
144,ZAR,1991,122.501517,24.110358,0.0
290,ZAR,1992,117.536158,24.069758,0.0
436,ZAR,1993,117.356483,23.846467,0.0
582,ZAR,1994,113.701350,24.098975,0.0
728,ZAR,1995,119.591717,24.466525,0.0
874,ZAR,1996,109.121400,24.239275,0.0
1020,ZAR,1997,127.673542,24.643383,0.0
1166,ZAR,1998,117.678642,24.657525,0.0
1312,ZAR,1999,125.678100,24.023367,0.0
1458,ZAR,2000,121.259100,24.291200,0.0


In [22]:
# Fix the difference of abbreviations for Rep. Dem. Congo
temp_rain_occur['Country']  = temp_rain_occur['Country'].replace('ZAR', 'COD')
temp_rain_occur[~temp_rain_occur['Country'].isin(countryList)]

,Country,Year,Precipitation,Temperature,Ocurrences


## Merge urbanization to the Main Table

In [23]:
urban= pd.merge(temp_rain_occur, urb,  how='left', left_on=['Country','Year'], right_on = ['Country','Year'])
urban.describe()

,Year,Precipitation,Temperature,Ocurrences,Urban (%),Rural (%)
count,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000
mean,2003.000000,150.946483,23.479929,10.657415,54.954128,45.045872
std,7.212084,187.992485,14.611533,180.138338,25.069653,25.069653
min,1991.000000,4.440862,-7.018926,0.000000,0.000000,0.000000
25%,1997.000000,75.340175,19.721479,0.000000,33.974000,24.218001
50%,2003.000000,125.000000,24.962717,0.000000,53.530000,46.470001
75%,2009.000000,186.939941,26.450129,0.000000,75.782000,66.025997
max,2015.000000,2744.000000,187.946183,5527.000000,100.000000,100.000000


## Normalization of the Country Code list

In [25]:
# Dataset of country names with country codes
countryCodes = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/CountryCodes.xlsx')
countryCodes = countryCodes.rename(columns={'Country ':'Country'})
countryCodes

,Country,Code
0,AFRICA,
1,Eastern Africa,
2,Burundi,BDI
3,Comoros,COM
4,Djibouti,DJI
5,Eritrea,ERI
6,Ethiopia,ETH
7,Kenya,KEN
8,Madagascar,MDG
9,Malawi,MWI


In [26]:
# Some country names diverge in different DataFrame and, therefore, the code of the country wasn't added.
# In order to fix it, the codes were manually concatened to the former list codes
d = {'Country': ['United States',
                 "CÙte d'Ivoire",
                 "Congo (Democratic Republic of the)",
                 "Korea (Republic of)",
                 "Tanzania (United Republic of)"],
     'Code': ['USA', 'CIV', 'COD', 'KOR', 'TZA']}
wrongCode = pd.DataFrame(data=d)

aux = [countryCodes, wrongCode]

countryCodes = pd.concat(aux)
countryCodes

,Country,Code
0,AFRICA,
1,Eastern Africa,
2,Burundi,BDI
3,Comoros,COM
4,Djibouti,DJI
5,Eritrea,ERI
6,Ethiopia,ETH
7,Kenya,KEN
8,Madagascar,MDG
9,Malawi,MWI


## Preprocessing of the HDI

In [27]:
# Dataset of the HDI
hdi = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/HDI.xlsx')
hdi['Country'] = hdi['Country'].str.strip()

In [28]:
hdi=pd.merge(hdi,countryCodes,
                   how='left',
                   left_on = ['Country'],
                   right_on = ['Country'])

# Even though some country names diverge and, moreover, the code of the country wasn't added,
# these contries don't belong to main DataFrame. Therefore, they're not relevant
hdi.loc[hdi['Code'].isnull()]

,HDI Rank (2017),Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Code
56,144,Eswatini (Kingdom of),0.536,0.537,0.536,0.531,0.524,0.520,0.513,0.504,...,0.527,0.538,0.550,0.561,0.572,0.580,0.584,0.586,0.588,NaN
74,7,"Hong Kong, China (SAR)",0.781,0.786,0.791,0.799,0.804,0.808,0.811,0.816,...,0.898,0.901,0.904,0.911,0.915,0.923,0.927,0.930,0.933,NaN
112,131,Micronesia (Federated States of),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.603,0.608,0.613,0.616,0.619,0.618,0.627,0.627,0.627,NaN
113,112,Moldova (Republic of),0.651,0.640,0.620,0.618,0.595,0.594,0.591,0.593,...,0.662,0.670,0.677,0.684,0.693,0.696,0.693,0.697,0.700,NaN
130,119,"Palestine, State of",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.670,0.672,0.677,0.687,0.679,0.679,0.687,0.689,0.686,NaN
168,80,The former Yugoslav Republic of Macedonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.731,0.735,0.738,0.740,0.743,0.747,0.754,0.756,0.757,NaN


In [29]:
# Drop useless columns
hdi.drop(columns=['HDI Rank (2017)','Country'],inplace=True)
hdi.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Code
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.453,0.463,0.471,0.482,0.487,0.491,0.493,0.494,0.498,AFG
1,0.645,0.626,0.610,0.613,0.619,0.632,0.641,0.641,0.652,0.662,...,0.729,0.741,0.752,0.767,0.771,0.773,0.776,0.782,0.785,ALB
2,0.577,0.581,0.587,0.591,0.595,0.600,0.608,0.617,0.627,0.636,...,0.719,0.729,0.736,0.740,0.745,0.747,0.749,0.753,0.754,DZA
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.830,0.828,0.827,0.849,0.850,0.853,0.854,0.856,0.858,AND
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.374,...,0.522,0.520,0.535,0.543,0.554,0.564,0.572,0.577,0.581,AGO


In [30]:
hdi.columns

Index([  1990,   1991,   1992,   1993,   1994,   1995,   1996,   1997,   1998,
         1999,   2000,   2001,   2002,   2003,   2004,   2005,   2006,   2007,
         2008,   2009,   2010,   2011,   2012,   2013,   2014,   2015,   2016,
         2017, 'Code'],
      dtype='object')

In [31]:
hdi.columns = hdi.columns.astype(str)
hdi.columns

Index(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', 'Code'],
      dtype='object')

In [32]:
# Adjust HDI to the format the main table it is set
values = hdi.loc[:, '1990':'2017']
hdiAdj = pd.melt(hdi, id_vars=['Code'], value_vars=values)
hdiAdj.head()

,Code,variable,value
0,AFG,1990,NaN
1,ALB,1990,0.645
2,DZA,1990,0.577
3,AND,1990,NaN
4,AGO,1990,NaN


In [33]:
hdiAdj.describe()

,value
count,4757.000000
mean,0.655067
std,0.165734
min,0.199000
25%,0.529000
50%,0.680000
75%,0.783000
max,0.953000


In [34]:
hdiAdj.dtypes

Code         object
variable     object
value       float64
dtype: object

In [35]:
# Rename to the format of the other tables
hdiAdj.rename(columns={'Code':'Country', 'variable':'Year', 'value':'HDI'}, inplace=True)
hdiAdj['Year'] = hdiAdj['Year'].astype(float)

In [36]:
# Select the timeframe of the main table
hdiAdj = hdiAdj.loc[(hdiAdj['Year']>=1991) & (hdiAdj['Year']<=2015)]
hdiAdj['Country'].isnull().sum()

150

## Merge HDI to the Main Table

In [85]:
# Check for countries of the main Dataframe that aren't in the HDI dataset
countryList = hdiAdj['Country'].unique()
urban['Country'].loc[~urban['Country'].isin(countryList)].unique()

array(['ABW', 'AIA', 'ASM', 'BMU', 'COK', 'CYM', 'GLP', 'GUF', 'GUM',
       'MAC', 'MNP', 'MSR', 'MTQ', 'MYT', 'NCL', 'NIU', 'NRU', 'PRI',
       'PYF', 'REU', 'SOM', 'TKL', 'TUV', 'VGB', 'WLF', 'TWN'],
      dtype=object)

In [136]:
data= pd.merge(urban, hdiAdj,  how='left', left_on=['Country','Year'], right_on = ['Country','Year'])
data.describe()

,Year,Precipitation,Temperature,Ocurrences,Urban (%),Rural (%),hdi
count,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000,2726.000000
mean,2003.000000,150.946483,23.479929,10.657415,54.954128,45.045872,0.645298
std,7.212084,187.992485,14.611533,180.138338,25.069653,25.069653,0.163042
min,1991.000000,4.440862,-7.018926,0.000000,0.000000,0.000000,0.208000
25%,1997.000000,75.340175,19.721479,0.000000,33.974000,24.218001,0.507000
50%,2003.000000,125.000000,24.962717,0.000000,53.530000,46.470001,0.673000
75%,2009.000000,186.939941,26.450129,0.000000,75.782000,66.025997,0.768000
max,2015.000000,2744.000000,187.946183,5527.000000,100.000000,100.000000,0.942000


## Preprocessing of the Population

In [87]:
pop = pd.read_csv('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/Population.csv')
#hdi['Country'] = hdi['Country'].str.strip()

In [88]:
pop.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.118
1,4,Afghanistan,2,Medium,1951,1951.5,4134.477,3705.031,7839.510
2,4,Afghanistan,2,Medium,1952,1952.5,4173.993,3760.979,7934.980
3,4,Afghanistan,2,Medium,1953,1953.5,4217.860,3820.747,8038.596
4,4,Afghanistan,2,Medium,1954,1954.5,4266.104,3884.348,8150.447


In [89]:
# Drop unnecessary columns and rename for the analyzes
pop.drop(columns=['VarID','LocID','Variant','MidPeriod','PopMale','PopFemale'],inplace=True)
pop.rename(columns={'Time':'Year', 'Location':'Country'}, inplace=True)
pop.head()

,Country,Year,PopTotal
0,Afghanistan,1950,7752.118
1,Afghanistan,1951,7839.510
2,Afghanistan,1952,7934.980
3,Afghanistan,1953,8038.596
4,Afghanistan,1954,8150.447


In [90]:
pop=pd.merge(pop,countryCodes,
                   how='left',
                   left_on = ['Country'],
                   right_on = ['Country'])

# Even though some country names diverge and, moreover, the code of the country wasn't added,
# these contries don't belong to main DataFrame. Therefore, they're not relevant
pop['Country'].loc[pop['Code'].isnull()].unique()

array(['Africa', 'Asia', 'Europe', 'High-income countries',
       'Latin America and the Caribbean', 'Least developed countries',
       'Less developed regions',
       'Less developed regions, excluding China',
       'Less developed regions, excluding least developed countries',
       'Low-income countries', 'Lower-middle-income countries',
       'Middle-income countries', 'More developed regions',
       'Northern America', 'Oceania', 'Sub-Saharan Africa',
       'Upper-middle-income countries', 'World'], dtype=object)

In [91]:
pop.head()

,Country,Year,PopTotal,Code
0,Afghanistan,1950,7752.118,AFG
1,Afghanistan,1951,7839.510,AFG
2,Afghanistan,1952,7934.980,AFG
3,Afghanistan,1953,8038.596,AFG
4,Afghanistan,1954,8150.447,AFG


In [92]:
# Rename to the format of the other tables
pop.drop(columns=['Country'],inplace=True)
pop.rename(columns={'Code':'Country'}, inplace=True)

In [93]:
# Select the timeframe of the main table
pop = pop.loc[(pop['Year']>=1991) & (pop['Year']<=2015)]
pop.head()

,Year,PopTotal,Country
41,1991,12993.657,AFG
42,1992,13981.231,AFG
43,1993,15095.099,AFG
44,1994,16172.719,AFG
45,1995,17099.541,AFG


In [97]:
pop.describe()

,Year,PopTotal
count,61425.000000,6.142500e+04
mean,2003.000000,1.985224e+05
std,7.211161,7.663329e+05
min,1991.000000,7.740000e-01
25%,1997.000000,5.288480e+02
50%,2003.000000,6.468827e+03
75%,2009.000000,3.468743e+04
max,2015.000000,7.383009e+06


In [99]:
# Drop duplicated lines
pop=pop.drop_duplicates()
pop.describe()

,Year,PopTotal
count,6825.000000,6.825000e+03
mean,2003.000000,1.985224e+05
std,7.211631,7.663828e+05
min,1991.000000,7.740000e-01
25%,1997.000000,5.288480e+02
50%,2003.000000,6.468827e+03
75%,2009.000000,3.468743e+04
max,2015.000000,7.383009e+06


## Merge Population to the Main Table

In [137]:
data= pd.merge(data, pop,  how='left', left_on=['Country','Year'], right_on = ['Country','Year'])
data.describe()

,Year,Precipitation,Temperature,Ocurrences,Urban (%),Rural (%),hdi,PopTotal
count,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000,2726.000000,3.675000e+03
mean,2003.000000,150.946483,23.479929,10.657415,54.954128,45.045872,0.645298,3.813460e+04
std,7.212084,187.992485,14.611533,180.138338,25.069653,25.069653,0.163042,1.445042e+05
min,1991.000000,4.440862,-7.018926,0.000000,0.000000,0.000000,0.208000,1.126000e+00
25%,1997.000000,75.340175,19.721479,0.000000,33.974000,24.218001,0.507000,3.869210e+02
50%,2003.000000,125.000000,24.962717,0.000000,53.530000,46.470001,0.673000,5.417070e+03
75%,2009.000000,186.939941,26.450129,0.000000,75.782000,66.025997,0.768000,2.273127e+04
max,2015.000000,2744.000000,187.946183,5527.000000,100.000000,100.000000,0.942000,1.397029e+06


## Preprocessing of the Education

In [118]:
# Dataset of the Education
education = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/education.xlsx')
education['Country'] = education['Country'].str.strip()
education

,HDI Rank (2017),Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,168,Afghanistan,0.122,0.133,0.145,0.156,0.168,0.179,0.190,0.202,...,0.347,0.359,0.371,0.382,0.390,0.398,0.406,0.410,0.410,0.415
1,68,Albania,0.584,0.589,0.558,0.543,0.542,0.551,0.558,0.571,...,0.646,0.652,0.673,0.693,0.730,0.732,0.733,0.733,0.742,0.745
2,85,Algeria,0.385,0.395,0.405,0.414,0.424,0.431,0.443,0.458,...,0.588,0.608,0.626,0.644,0.652,0.660,0.661,0.662,0.663,0.664
3,35,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.661,0.661,0.662,0.663,0.715,0.715,0.716,0.715,0.714,0.714
4,147,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.390,0.404,0.398,0.423,0.435,0.447,0.460,0.472,0.487,0.498
5,70,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.674,0.672,0.667,0.659,0.663,0.669,0.669,0.676,0.676,0.676
6,47,Argentina,0.628,0.632,0.634,0.636,0.639,0.648,0.659,0.670,...,0.753,0.775,0.802,0.807,0.805,0.804,0.808,0.812,0.814,0.816
7,83,Armenia,0.634,0.637,0.636,0.642,0.636,0.632,0.628,0.641,...,0.712,0.729,0.731,0.732,0.735,0.739,0.742,0.746,0.749,0.749
8,3,Australia,0.873,0.874,0.874,0.875,0.875,0.894,0.895,0.895,...,0.911,0.909,0.913,0.916,0.919,0.921,0.924,0.926,0.929,0.929
9,20,Austria,0.676,0.683,0.692,0.694,0.705,0.709,0.712,0.715,...,0.809,0.819,0.837,0.838,0.841,0.833,0.844,0.847,0.852,0.852


In [119]:
education=pd.merge(education,countryCodes,
                   how='left',
                   left_on = ['Country'],
                   right_on = ['Country'])

# Even though some country names diverge and, moreover, the code of the country wasn't added,
# these contries don't belong to main DataFrame. Therefore, they're not relevant
education.loc[education['Code'].isnull()]

,HDI Rank (2017),Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Code
56,144,Eswatini (Kingdom of),0.407,0.412,0.413,0.412,0.410,0.408,0.404,0.401,...,0.491,0.501,0.508,0.515,0.522,0.528,0.528,0.528,0.528,NaN
74,7,"Hong Kong, China (SAR)",0.640,0.645,0.650,0.655,0.660,0.665,0.669,0.673,...,0.807,0.808,0.807,0.824,0.830,0.846,0.852,0.855,0.855,NaN
112,131,Micronesia (Federated States of),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.552,0.564,0.573,0.581,0.590,0.590,0.590,0.592,0.590,NaN
113,112,Moldova (Republic of),0.598,0.597,0.596,0.595,0.594,0.593,0.586,0.590,...,0.697,0.696,0.699,0.709,0.716,0.714,0.709,0.710,0.710,NaN
130,119,"Palestine, State of",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.651,0.655,0.660,0.667,0.654,0.655,0.657,0.660,0.660,NaN
168,80,The former Yugoslav Republic of Macedonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.657,0.662,0.667,0.670,0.673,0.676,0.689,0.691,0.691,NaN


In [120]:
# Drop useless columns
education.drop(columns=['HDI Rank (2017)','Country'],inplace=True)
education.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Code
0,0.122,0.133,0.145,0.156,0.168,0.179,0.190,0.202,0.213,0.224,...,0.359,0.371,0.382,0.390,0.398,0.406,0.410,0.410,0.415,AFG
1,0.584,0.589,0.558,0.543,0.542,0.551,0.558,0.571,0.580,0.586,...,0.652,0.673,0.693,0.730,0.732,0.733,0.733,0.742,0.745,ALB
2,0.385,0.395,0.405,0.414,0.424,0.431,0.443,0.458,0.473,0.487,...,0.608,0.626,0.644,0.652,0.660,0.661,0.662,0.663,0.664,DZA
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.661,0.662,0.663,0.715,0.715,0.716,0.715,0.714,0.714,AND
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276,...,0.404,0.398,0.423,0.435,0.447,0.460,0.472,0.487,0.498,AGO


In [124]:
education.columns

Index([  1990,   1991,   1992,   1993,   1994,   1995,   1996,   1997,   1998,
         1999,   2000,   2001,   2002,   2003,   2004,   2005,   2006,   2007,
         2008,   2009,   2010,   2011,   2012,   2013,   2014,   2015,   2016,
         2017, 'Code'],
      dtype='object')

In [125]:
education.columns = education.columns.astype(str)
education.columns

Index(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', 'Code'],
      dtype='object')

In [126]:
# Adjust education to the format the main table it is set
values = education.loc[:, '1990':'2017']
educationAdj = pd.melt(education, id_vars=['Code'], value_vars=values)
educationAdj.head()

,Code,variable,value
0,AFG,1990,0.122
1,ALB,1990,0.584
2,DZA,1990,0.385
3,AND,1990,NaN
4,AGO,1990,NaN


In [127]:
educationAdj.describe()

,value
count,4773.000000
mean,0.577983
std,0.187790
min,0.081000
25%,0.436000
50%,0.609000
75%,0.714000
max,0.941000


In [128]:
educationAdj.dtypes

Code         object
variable     object
value       float64
dtype: object

In [129]:
# Rename to the format of the other tables
educationAdj.rename(columns={'Code':'Country', 'variable':'Year', 'value':'Education'}, inplace=True)
educationAdj['Year'] = educationAdj['Year'].astype(float)

In [131]:
# Select the timeframe of the main table
educationAdj = educationAdj.loc[(educationAdj['Year']>=1991) & (educationAdj['Year']<=2015)]
educationAdj['Country'].isnull().sum()
# The same amount as in HDI

150

## Merge education to the main table

In [134]:
# Check for countries of the main Dataframe that aren't in the HDI dataset
countryList = educationAdj['Country'].unique()
urban['Country'].loc[~urban['Country'].isin(countryList)].unique()

array(['ABW', 'AIA', 'ASM', 'BMU', 'COK', 'CYM', 'GLP', 'GUF', 'GUM',
       'MAC', 'MNP', 'MSR', 'MTQ', 'MYT', 'NCL', 'NIU', 'NRU', 'PRI',
       'PYF', 'REU', 'SOM', 'TKL', 'TUV', 'VGB', 'WLF', 'TWN'],
      dtype=object)

In [138]:
data= pd.merge(data, educationAdj,  how='left', left_on=['Country','Year'], right_on = ['Country','Year'])
data.describe()

,Year,Precipitation,Temperature,Ocurrences,Urban (%),Rural (%),hdi,PopTotal,education
count,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000,3675.000000,2726.000000,3.675000e+03,2738.000000
mean,2003.000000,150.946483,23.479929,10.657415,54.954128,45.045872,0.645298,3.813460e+04,0.563970
std,7.212084,187.992485,14.611533,180.138338,25.069653,25.069653,0.163042,1.445042e+05,0.185258
min,1991.000000,4.440862,-7.018926,0.000000,0.000000,0.000000,0.208000,1.126000e+00,0.125000
25%,1997.000000,75.340175,19.721479,0.000000,33.974000,24.218001,0.507000,3.869210e+02,0.416000
50%,2003.000000,125.000000,24.962717,0.000000,53.530000,46.470001,0.673000,5.417070e+03,0.589000
75%,2009.000000,186.939941,26.450129,0.000000,75.782000,66.025997,0.768000,2.273127e+04,0.702000
max,2015.000000,2744.000000,187.946183,5527.000000,100.000000,100.000000,0.942000,1.397029e+06,0.940000


In [ ]:
factors=[]
for x in hdi_complete['Country'].unique():

    lCountry = []
    lCountry.append(x)        
    country = hdi_complete.loc[hdi_complete["Country"] == x]
    value = country[:,['Precipitation','Temperature','hdi']]
    value


In [ ]:
for x in hdi_complete['Country'].unique():
    print(x)

In [ ]:
aux=t[['Year','Precipitation','Temperature']]
aux.set_index('Year',inplace=True)
aux

In [ ]:
j = pd.DataFrame(np.diff(aux))
#t.dtypes
j